# 2.4 Maxwell's Equations

[Peter Monk: "Finite Elements for Maxwell's Equations"]

## Magnetostatic field generated by a permanent magnet

magnetic flux $B$, magnetic field $H$, given magnetization $M$:

$$
\DeclareMathOperator{\Grad}{grad}
\DeclareMathOperator{\Curl}{curl}
\DeclareMathOperator{\Div}{div}
B = \mu (H + M), \quad \Div B = 0, \quad \Curl H = 0
$$

Introducing a vector-potential $A$ such that $B = \Curl A$, and putting equations together we get

$$
\Curl \mu^{-1} \Curl A = \Curl M
$$

In weak form: Find $A \in H(\Curl)$ such that

$$
\int \mu^{-1} \Curl A \Curl v = \int M \Curl v \qquad  \forall \, v \in H(\Curl)
$$

Usually, the permeability $\mu$ is given as 
$\mu = \mu_r \mu_0$, with $\mu_0 = 4 \pi 10^{-7}$ the permeability of vacuum.

In [61]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

Geometric model and meshing of a bar magnet:

In [62]:
# box = OrthoBrick(Pnt(-3,-3,-3),Pnt(3,3,3)).bc("outer")
# magnet = Cylinder(Pnt(-1,0,0),Pnt(1,0,0), 0.3) * OrthoBrick(Pnt(-1,-3,-3),Pnt(1,3,3))
# air = box - magnet
box = Box( (-3,-3,-3), (3,3,3))
box.faces.name = "outer"

magnet = Cylinder((-1,0,0),X, r=0.3, h=2)
magnet.mat("magnet")
magnet.faces.col = (1,0,0)

air = box-magnet
air.mat("air")
shape = Glue([air,magnet])
geo = OCCGeometry(shape)

Draw (shape, clipping={ "z" : -1, "function":True})
                       
mesh = Mesh(geo.GenerateMesh(maxh=2, curvaturesafety=1))
mesh.Refine()
# mesh.Refine()
mesh.Curve(3);



WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

In [63]:
mesh.GetMaterials(), mesh.GetBoundaries(), mesh.ne

(('air', 'magnet'),
 ('outer',
  'outer',
  'outer',
  'outer',
  'outer',
  'outer',
  'default',
  'default',
  'default'),
 18440)

In [64]:
class MGPreconditioner(BaseMatrix):
    def __init__ (self, fes, level, mat, coarsepre):
        super().__init__()
        self.fes = fes
        self.level = level
        self.mat = mat
        self.coarsepre = coarsepre
        if level > 0:
            self.localpre = mat.CreateSmoother(fes.FreeDofs())
        else:
            self.localpre = mat.Inverse(fes.FreeDofs())

    def Mult (self, d, w):
        if self.level == 0:
            w.data = self.localpre * d
            return

        prol = self.fes.Prolongation().Operator(self.level)

        w[:] = 0
        self.localpre.Smooth(w,d)
        res  = d - self.mat * w
        w += prol @ self.coarsepre @ prol.T * res
        self.localpre.SmoothBack(w,d)


    def Shape (self):
        return self.localpre.shape
    def CreateVector (self, col):
        return self.localpre.CreateVector(col)

Define space, forms and preconditioner.

* To obtain a regular system matrix, we regularize by adding a very small $L_2$ term.
* We solve magnetostatics, so we can gauge by adding and arbitrary gradient field. A cheap possibility is to delete all basis-functions which are gradients (flag 'nograds')

In [65]:
order = 3

fes = HCurl(mesh, order=order, dirichlet="outer", nograds=True)
print ("ndof =", fes.ndof)
u,v = fes.TnT()


from math import pi
mu0 = 4*pi*1e-7
mur = mesh.MaterialCF({"magnet" : 1000}, default=1)

a = BilinearForm(fes)

ir = IntegrationRule(fem.ET.TET, order = 3*order)

a += SymbolicBFI(1/(mu0*mur)*curl(u)*curl(v) + 1e-8/(mu0*mur)*u*v, intrule = ir)
# c = Preconditioner(a, "bddc")
c = Preconditioner(a, "multigrid", blocktype = "ed")

f = LinearForm(fes)
mag = mesh.MaterialCF({"magnet" : (1,0,0)}, default=(0,0,0))
f += mag*curl(v) * dx("magnet")

ndof = 266000


: 

Assemble system and setup preconditioner using task-parallelization:

In [66]:
with TaskManager():
    a.Assemble()
    f.Assemble()
    pre = MGPreconditioner(fes, 0, a.mat, None)

In [ ]:
help(Preconditioner)

Finally, declare GridFunction and solve by preconditioned CG iteration:

In [ ]:
gfu = GridFunction(fes)
with TaskManager():
    solvers.CG(sol=gfu.vec, rhs=f.vec, mat=a.mat, pre=c.mat, printrates=True)

CG iteration 1, residual = 0.002538501669103207     
CG iteration 2, residual = 0.0004588664851979842     
CG iteration 3, residual = 0.00016375383401061747     
CG iteration 4, residual = 7.053305933340474e-05     
CG iteration 5, residual = 3.988214122176495e-05     
CG iteration 6, residual = 2.2443292149005436e-05     
CG iteration 7, residual = 1.5266513779665523e-05     
CG iteration 8, residual = 9.79781951707914e-06     
CG iteration 9, residual = 6.565640561209574e-06     
CG iteration 10, residual = 4.573582203479596e-06     
CG iteration 11, residual = 3.3146413674793955e-06     
CG iteration 12, residual = 2.4396544789741577e-06     
CG iteration 13, residual = 1.8839454101536006e-06     
CG iteration 14, residual = 1.522250954492012e-06     
CG iteration 15, residual = 1.2534985282997951e-06     
CG iteration 16, residual = 1.0479518001479216e-06     
CG iteration 17, residual = 9.319419003486321e-07     
CG iteration 18, residual = 8.797897725392224e-07     
CG iteration 

In [ ]:
Draw (curl(gfu), mesh, "B-field", draw_surf=False, \
      clipping = { "z" : -1, "function":True}, \
      vectors = { "grid_size":50}, min=0, max=2e-5);

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…